# Heat Case Study - Energy System Model

In [ ]:
import os

import pandas as pd
from datapackage import Package 

import oemof.tabular.facades as fc
from oemof.solph import EnergySystem, Model, Bus, Flow, Transformer
from oemof.solph.components import GenericStorage
from oemof.tabular.tools import postprocessing as pp


results_path = "results"

if not os.path.exists(results_path): 
    os.mkdir(results_path)
    
scenario_path = os.path.join(results_path, "base")

if not os.path.exists(scenario_path): 
    os.mkdir(scenario_path)

In [ ]:
timeseries = pd.read_csv("data/timeseries.csv", parse_dates=True, index_col=0)

data = pd.DataFrame(Package("data/heating-system/datapackage.json").
                    get_resource("heating-system").
                    read(keyed=True)).set_index(["plant", "parameter", "unit"])


timeseries.index.freq = "H"

In [ ]:
data.unstack(['plant']).fillna('-')

## Create energy system

In [ ]:
es = EnergySystem(timeindex=timeseries.index)

## Add Components to energy system

### Bus Constraint

With the set of all Buses $B$ all inputs $x^{flow}_{i(b),b}$ to a bus $b$ must equal all its outputs $x^{flow}_{b,o(b)}$

$$\sum_i x^{flow}_{i(b), b}(t) - \sum_o x^{flow}_{b, o(b)}(t) = 0 \qquad \forall t \in T, \forall b \in B$$

Every time a `Component` is created, the connected bus inputs /outputs will be updated. By this update every bus has all required information of its inputs and outputs available to construct the constraints. The equations will be build by running `Model(es)` once the complete energy system is setup with its components.

For the case study four buses will be modeled: 

1. *Electricity Bus* which represents the electricty grid
2. *Heat Bus* which represents the district heating system (grid)
3. *Heat Bus Low* which is used to model the low temperature heat that comes from the storage 
4. *Gas Bus* for the commodity used by fossil fuel based components

In [ ]:
elec_bus = Bus(label="elec_bus")
heat_bus = Bus(label="heat-bus")
heat_bus_low = Bus(label="heat-bus-low")
gas_bus = Bus(label="gas-bus")

es.add(elec_bus, heat_bus, gas_bus, heat_bus_low)

### Heat Load

For the set of all Load denoted with $l \in L$ the load $x_l$ at timestep t equals the exogenously defined  profile value $c^{profile}_l$ multiplied by the amount of this load $c^{amount}_l$

$$ x^{flow}_{l}(t) = c^{profile}_{l}(t) \cdot c^{amount}_{l} \qquad \forall t \in T, \forall l \in L$$



In [ ]:
es.add(
    fc.Load(
        label="heat-load",
        bus=heat_bus,
        amount=1145.1273e3,
        profile=timeseries["heat_load_profile"]))



### Excess

This component is used to model the grid. 

$$ x^{flow}_{e}(t) \leq \inf \qquad \forall t \in T, \forall e \in E$$

The marinal costs are added with a negative sign, as all produced electricity represents revenues for the heating system.

In [ ]:
es.add(
    fc.Excess(
        label="to_grid",
        bus=elec_bus,
        marginal_cost=-1 * timeseries["electricity_price"].values))

es.add(
    fc.Shortage(
        label="from_grid",
        tech="grid",
        carrier="electricity",
        bus=elec_bus,
        marginal_cost=timeseries["electricity_price"].values))

### Commodity

The commodity object will bound the flow from a source to a bus by a certain `amount` by the following equation:

$$ \sum_{t} x^{flow}(t) \leq c^{amount} \qquad for t \in T$$



In [ ]:
es.add(
    fc.Commodity(
        label="gas-commdity",
        bus=gas_bus,
        amount=10e10,
        carrier="gas",
        tech="commodity",
        marginal_cost=float(data.loc[("gas", "cost", slice(None)), "value"]),
    )
)

### Waste Heat Source

In [ ]:
es.add(
    fc.Dispatchable(
        label="mustrun",
        bus=heat_bus,
        capacity=float(data.loc[("mustrun", "qmax_pmax", slice(None)), "value"]),
        carrier="heat",
        tech="waste",
        marginal_cost=0,
    )
)

### Extraction Turbine Unit

The mathematical description is derived from the oemof base class
[ExtractionTurbineCHP](https://oemof.readthedocs.io/en/
stable/oemof_solph.html#extractionturbinechp-component>):

$$x^{flow, carrier}(t) =
    \frac{x^{flow, electricity}(t) + x^{flow, heat}(t) \
    \cdot c^{beta}(t)}{c^{condensing\_efficiency}(t)}
    \qquad \forall t \in T$$

$$
    x^{flow, electricity}(t)  \geq  x^{flow, thermal}(t) \cdot
    \frac{c^{electrical\_efficiency}(t)}{c^{thermal\_efficiency}(t)}
    \qquad \forall t \in T$$

where $c^{beta}$ is defined as:

$$
    c^{beta}(t) = \frac{c^{condensing\_efficiency}(t) -
    c^{electrical\_efficiency(t)}}{c^{thermal\_efficiency}(t)}
    \qquad \forall t \in T$$

**Objective expression** for operation includes marginal cost and/or
carrier costs:

$$
        x^{opex} = \sum_t (x^{flow, out}(t) \cdot c^{marginal\_cost}(t) +
        x^{flow, carrier}(t) \cdot c^{carrier\_cost}(t))
$$


In [ ]:
es.add(
    fc.ExtractionTurbine(
        label="extraction-turbine",
        electricity_bus=elec_bus,
        heat_bus=heat_bus,
        fuel_bus=gas_bus,
        carrier="gas",
        tech="ext",
        capacity=float(data.loc[
            ("ext", "pmax_qmin", slice(None)), "value"]),
        condensing_efficiency=float(data.loc[
            ("ext", "eta_el_pmax_qmin", slice(None)), "value"]),
        electric_efficiency=float(data.loc[
            ("ext", "eta_el_pmax_qmax", slice(None)), "value"]),
        thermal_efficiency=float(data.loc[
            ("ext", "eta_th_qmax_pmax", slice(None)), "value"])
    )
)

### Backpressure Units / Motoric CHP 

Backpressure turbine power plants are modelled with a constant relation
between heat and electrical output (power to heat coefficient).

$$ x^{flow, carrier}(t) =
    \frac{x^{flow, electricity}(t) + x^{flow, heat}(t)}\
    {c^{th\_efficiency}(t) + c^{el\_efficiency}(t)}
    \qquad \forall t \in T $$


$$    \frac{x^{flow, electricity}(t)}{x_{flow, thermal}(t)} =
    \frac{c^{el\_efficiency}(t)}{c^{th\_efficiency}(t)}
    \qquad \forall t \in T $$

**Objective expression** for operation includes marginal cost and/or
carrier costs:

$$        x^{opex} = \sum_t (x^{flow, out}(t) \cdot c^{marginal\_cost}(t) +
          x^{flow, carrier}(t) \cdot c^{carrier\_cost}(t))$$

In [ ]:
es.add(
    fc.BackpressureTurbine(
        label="motoric-chp",
        electricity_bus=elec_bus,
        heat_bus=heat_bus,
        fuel_bus=gas_bus,
        carrier="gas",
        tech="bp",
        capacity=float(data.loc[
            ("bp", "pmax_qmin", slice(None)), "value"]),
        electric_efficiency=float(data.loc[
            ("bp", "eta_el_pmax_qmax", slice(None)), "value"]),
        thermal_efficiency=float(data.loc[
            ("bp", "eta_th_qmax_pmax", slice(None)), "value"])
    )
)

### Storage

The mathematical representation of the storage for all storages $s \in S$ will include the flow into the storage, out of the storage and a storage level. The default efficiency for input/output is 1. Note that this is is included during charge and discharge. If you want to set the round trip efficiency you need to do for example: $\eta = \sqrt{\eta^{roundtrip}}$

Intertemporal energy balance of the storage:

$$ x^{level}_{s}(t) = \eta^{loss} x^{level}_{s}(t) + \eta x^{flow}_{s, in} - \eta x^{flow}_{s, out}(t) \qquad \forall t \in T,  \forall s \in S$$ 

Bounds of the storage level variable $x^{level}_s(t)$:

$$ x^{level}_s(t) \leq c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$


$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$ 

Of course, in addition the inflow/outflow of the storage also needs to be within the limit of the minimum and maximum power. 

$$ -c_s^{capacity} \leq x^{flow}_s(t) \leq c_s^{capacity} \qquad \forall t \in T, \forall s \in S$$ 





In [ ]:
if True:
    storage = fc.Storage(
            label="heat-storage",
            bus=heat_bus,
            carrier="water",
            tech="storage",
            capacity=float(data.loc[("storage", "qmax_in", slice(None)), "value"]),
            storage_capacity=float(data.loc[("storage", "umax", slice(None)), "value"]),
            balanced=True, # oemof.solph argument
            initial_storage_level=0.5, # oemof.solph argument
            max_storage_level=1, # oemof.solph argument
    )

    # the output of the storage needs to be connected to the low temperature heat bus 
    output_edge = storage.outputs[heat_bus]
    storage.outputs.pop(heat_bus)
    storage.outputs.update({heat_bus_low: output_edge})

    es.add(storage)

In [ ]:
# not used 
if False:
    es.add(
        GenericStorage(
            label="heat-storage",
            inputs={
                heat_bus: Flow(
                    nominal_value=float(
                        data.loc[("storage", "qmax_in", slice(None)), "value"]
                    )
                )
            },
            outputs={
                heat_bus_low: Flow(
                    nominal_value=float(
                        data.loc[("storage", "qmax_in", slice(None)), "value"]
                    )
                )
            },
            loss_rate=0,
            nominal_storage_capacity=float(
                data.loc[("storage", "umax", slice(None)), "value"]
            ),
            inflow_conversion_factor=float(
                data.loc[("storage", "eta_in", slice(None)), "value"]
            ),
            outflow_conversion_factor=float(
                data.loc[("storage", "eta_in", slice(None)), "value"]
            ),
        )
    )


#### Heat Pump

The storage is not directly connected to the heat bus but the heat will be lifted to higher temperature levels with a heat pump. 

$$ x_{elec\_bus, hp}^{flow} = \frac{1}{c^{COP}} \cdot x_{hp, heat\_bus}^{flow}$$

$$ x_{storage, heat\_bus\_low}^{flow} = x_{hp, heat\_bus}^{flow} \frac{c^{COP} -1}{c^{COP}}$$

In [ ]:
cop = float(data.loc[("storage", "cop", slice(None)), "value"])
# not used 
if False:
    es.add(
        Transformer(
            label="hp-storage",
            inputs={elec_bus: Flow(), heat_bus_low: Flow()},
            outputs={heat_bus: Flow()},
            conversion_factors={elec_bus: 1 / cop, heat_bus_low: (cop - 1) / cop},
        )
    )
if True:
    es.add(
            fc.HeatPump(
            label="hp-storage",
            carrier="electricity",
            capacity=float(data.loc[("storage", "qmax_out", slice(None)), "value"]),
            tech="hp",
            cop=float(data.loc[("storage", "cop", slice(None)), "value"]), 
            electricity_bus=elec_bus,
            high_temperature_bus=heat_bus, 
            low_temperature_bus=heat_bus_low
        )
    )

### Heat pump and electro boiler 

To model the heat pump as well as the boiler, a conversion unit is used. This object will take from a bus and feed into another and is represented by the following mathematical equation: 

$$x^{flow}_{c, to}(t) = c^{efficiencty}_{c} \cdot x^{flow}_{c, from}(t), \qquad \forall c  \in C, \forall t \in T$$ 

The `capacity` refers to the output of the conversion unit. Thus, to the thermal capacity of the heat pump / boiler. Therefore the flow from the heat pump to the heating system is bounded by the thermal capacity of the repective unit:

$$x^{flow}_{c, to}(t) \leq c^{capacity}_{c, to}(t) \qquad \forall c  \in C, \forall t \in T$$



In [ ]:
es.add(
    fc.Conversion(
        label="heat-pump",
        from_bus=elec_bus,
        to_bus=heat_bus,
        carrier="electricity",
        tech="hp",
        capacity=float(data.loc[
            ("heatpump", "q_max", slice(None)), "value"]),
        efficiency=(float(data.loc[
            ("heatpump", "cop_max", slice(None)), "value"]) + 
                    float(data.loc[
            ("heatpump", "cop_min", slice(None)), "value"])) / 2,
    )
)

es.add(
    fc.Conversion(
        label="electro-boiler",
        from_bus=elec_bus,
        to_bus=heat_bus,
        carrier="electricity",
        tech="eboiler",
        capacity=float(data.loc[
            ("eboiler", "q_max", slice(None)), "value"]),
        efficiency=float(data.loc[
            ("eboiler", "eta_th_max", slice(None)), "value"]),
    )
)

### Objective Function 

The objective function is created from all instantiated objects. It will use all operating costs (i.e. `marginal_cost` argument) and if set all investment costs (i.e. `capacity_cost` argument)

$$ \text{min:} \sum_g \sum_t \overbrace{c^{marginal\_cost}_g \cdot x^{flow}_{g}(t)}^{\text{operating cost}} \\ 
\sum_g \sum_t \overbrace{c^{capacity\_cost}_g \cdot x^{capacity}_{g}(t)}^{\text{investment cost}} $$

**Note**: In this model the selling of electricity to the market is represented through negativ marginal cost of the excess component.

In [ ]:
m = Model(es)
  
#m.write(os.path.join(scenario_path, 'model.lp'), io_options={'symbolic_solver_labels': True})

In [ ]:
m.solve(solver="cbc")

m.results = m.results()

In [ ]:
from oemof.outputlib import views
df = pd.DataFrame()
for b in [heat_bus, heat_bus_low, elec_bus, gas_bus]:
    df = pd.concat([
        df, 
        views.node(m.results, b, multiindex=True).get("sequences")
    ], axis=1)
df.round(7).to_csv(os.path.join(scenario_path, 'endogenous-variables.csv'))

In [ ]:
r = (
    df[es.groups['hp-storage'], heat_bus, 'flow'] - 
    df[heat_bus, es.groups['heat-storage'], 'flow']).reset_index()
r.columns = ['timeindex', 'storage']
s = views.node(m.results, es.groups['heat-storage'], multiindex=True)['sequences']

In [ ]:
s.plot()

In [ ]:
s.to_csv(os.path.join(scenario_path, 'storage-variables.csv'))

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.offline as py_offline
#fig = go.Figure()
#fig.add_trace())
#fig.show()
data = [
    go.Scatter(
        x=r.timeindex,
        y=r.storage
    )
]

py_offline.plot(data, filename=os.path.join(scenario_path, 'basic-line'))
